# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.  
 
Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
import pandas as pd

# Define the column names
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']

# Load the first 20,000 rows of the CSV file into a DataFrame with specified column names
subset_data = pd.read_csv('Sentiment140.csv', nrows=20000, header=None, names=column_names, skiprows=1)

# Display basic information about the subset DataFrame
display("Number of records in the subset:", len(subset_data))
display("Columns in the subset:", subset_data.columns)
display("Sample of the subset data:")
display(subset_data.head())


'Number of records in the subset:'

20000

'Columns in the subset:'

Index(['target', 'id', 'date', 'flag', 'user', 'text'], dtype='object')

'Sample of the subset data:'

,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [2]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import nltk


# Load a subset of the CSV file into a DataFrame
subset_data = pd.read_csv('Sentiment140.csv', nrows=20000, header=None)  # Specify header=None for no header

# Define the column names
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']

# Assign column names to the DataFrame
subset_data.columns = column_names

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Define the functions

def clean_up(s):
    # Remove special characters, numbers, and URLs
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', s)
    cleaned_text = re.sub(r'http\S+', '', cleaned_text)
    return cleaned_text.lower()

def tokenize(s):
    # Tokenize a string
    return word_tokenize(s)

def stem_and_lemmatize(l):
    # Perform stemming and lemmatization on a list of words
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    stemmed_and_lemmatized = []
    for word in l:
        stemmed_word = stemmer.stem(word)
        lemmatized_word = lemmatizer.lemmatize(word)
        stemmed_and_lemmatized.append(lemmatized_word)
    return stemmed_and_lemmatized

def remove_stopwords(l):
    # Remove English stopwords from a list of strings
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in l if word not in stop_words]
    return filtered_words

# Apply the functions to the 'text' column and create a new 'text_processed' column
subset_data['text_processed'] = subset_data['text'].apply(clean_up)
subset_data['text_processed'] = subset_data['text_processed'].apply(tokenize)
subset_data['text_processed'] = subset_data['text_processed'].apply(stem_and_lemmatize)
subset_data['text_processed'] = subset_data['text_processed'].apply(remove_stopwords)

# Display a sample of the processed data
display(subset_data[['target', 'id', 'date', 'flag', 'user', 'text_processed']].head())


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\benoi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\benoi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\benoi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,target,id,date,flag,user,text_processed
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"[switchfoot, awww, thats, bummer, shoulda, got..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,"[upset, cant, update, facebook, texting, might..."
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,"[kenichan, dived, many, time, ball, managed, s..."
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,"[whole, body, feel, itchy, like, fire]"
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"[nationwideclass, behaving, im, mad, cant, see]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [3]:
from nltk.probability import FreqDist

# Combine all words from the 'text_processed' column
all_words = [word for sublist in subset_data['text_processed'] for word in sublist]

# Calculate the frequency distribution of all words
freq_dist = FreqDist(all_words)

# Select the top 5,000 words
top_words = freq_dist.most_common(5000)

display(top_words)


[('im', 2456),
 ('wa', 1487),
 ('day', 1276),
 ('work', 1258),
 ('go', 1228),
 ('get', 1182),
 ('dont', 1083),
 ('cant', 1075),
 ('got', 980),
 ('like', 953),
 ('today', 943),
 ('going', 886),
 ('want', 844),
 ('back', 801),
 ('time', 783),
 ('still', 747),
 ('really', 726),
 ('one', 690),
 ('good', 687),
 ('need', 637),
 ('know', 635),
 ('wish', 621),
 ('ha', 620),
 ('miss', 615),
 ('sad', 615),
 ('night', 603),
 ('u', 587),
 ('home', 585),
 ('feel', 580),
 ('last', 573),
 ('think', 564),
 ('sorry', 542),
 ('na', 534),
 ('didnt', 515),
 ('bad', 514),
 ('much', 509),
 ('sleep', 500),
 ('oh', 495),
 ('well', 490),
 ('lol', 484),
 ('see', 452),
 ('hate', 450),
 ('love', 442),
 ('tomorrow', 432),
 ('sick', 431),
 ('hope', 423),
 ('twitter', 408),
 ('make', 400),
 ('morning', 395),
 ('new', 387),
 ('bed', 387),
 ('amp', 387),
 ('though', 382),
 ('getting', 363),
 ('would', 363),
 ('week', 357),
 ('come', 348),
 ('right', 347),
 ('tonight', 347),
 ('could', 342),
 ('hour', 337),
 ('thing', 

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [4]:
from nltk.probability import FreqDist

# Combine all words from the 'text_processed' column
all_words = [word for sublist in subset_data['text_processed'] for word in sublist]

# Calculate the frequency distribution of all words
freq_dist = FreqDist(all_words)

# Select the top 5,000 words
top_words = [word for word, _ in freq_dist.most_common(5000)]

# Define a function to find features for a document
def find_features(document):
    words = set(document)
    features = {}
    for w in top_words:
        features[w] = (w in words)
    return features

# Create the feature set as a list of tuples
feature_set = []
for text in subset_data['text_processed']:
    features = find_features(text)
    feature_set.append((features, True))  # We set all labels as True (positive) for now

# Display a sample of the feature set
display(feature_set[:4])


[({'im': False,
   'wa': False,
   'day': True,
   'work': False,
   'go': False,
   'get': False,
   'dont': False,
   'cant': False,
   'got': True,
   'like': False,
   'today': False,
   'going': False,
   'want': False,
   'back': False,
   'time': False,
   'still': False,
   'really': False,
   'one': False,
   'good': False,
   'need': False,
   'know': False,
   'wish': False,
   'ha': False,
   'miss': False,
   'sad': False,
   'night': False,
   'u': False,
   'home': False,
   'feel': False,
   'last': False,
   'think': False,
   'sorry': False,
   'na': False,
   'didnt': False,
   'bad': False,
   'much': False,
   'sleep': False,
   'oh': False,
   'well': False,
   'lol': False,
   'see': False,
   'hate': False,
   'love': False,
   'tomorrow': False,
   'sick': False,
   'hope': False,
   'twitter': False,
   'make': False,
   'morning': False,
   'new': False,
   'bed': False,
   'amp': False,
   'though': False,
   'getting': False,
   'would': False,
   'week': F

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [6]:
import random

# Split the feature set into training and testing sets
train_size = int(0.8 * len(feature_set))  # 80% for training, 20% for testing
train_set = feature_set[:train_size]
test_set = feature_set[train_size:]

# Train the NaiveBayesClassifier
classifier = nltk.NaiveBayesClassifier.train(train_set)

# Show the most informative features
classifier.show_most_informative_features(15)


Most Informative Features


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [7]:
# Split the feature set into training and test sets
train_size = int(0.8 * len(feature_set))
training_set = feature_set[:train_size]
testing_set = feature_set[train_size:]

# Create and train the Naive Bayes classifier
classifier = nltk.NaiveBayesClassifier.train(training_set)

# Test the classifier using the testing set
accuracy = nltk.classify.accuracy(classifier, testing_set)
print("Classifier accuracy percent:", accuracy * 100)


Classifier accuracy percent: 100.0


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here